In [ ]:
# creating the path (PYTHONPATH) to our module.
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pyscf import scf, gto
from pyscf import lib
from pyscf import ao2mo
import numpy as np

#mol = gto.M(atom='''
#            C1       0.0000000000            0.4663393949            0.5932864698
#            C2       0.0000000000           -0.4663393949           -0.5932864698
#            H1       0.8884363804            1.0959623133            0.5921146257
#            H2      -0.8884363804            1.0959623133            0.5921146257
#            H3       0.8884363804           -1.0959623133           -0.5921146257
#            H4      -0.8884363804           -1.0959623133           -0.5921146257
#            F1       0.0000000000           -0.2947283454            1.7313694084
#            F2       0.0000000000            0.2947283454           -1.7313694084
#''', basis = 'cc-pvdz', unit='A')
mol = gto.M(atom='Br 0 0 0; H 0 0 1', basis='cc-pvdz')
mf = scf.RHF(mol)
mf.kernel()



mo_energy = mf.mo_energy
occidx = np.where(mf.mo_occ > 0)[0]
viridx = np.where(mf.mo_occ == 0)[0]
orbv = mf.mo_coeff[:, viridx]
orbo = mf.mo_coeff[:, occidx]
nvir = orbv.shape[1]
nocc = orbo.shape[1]

nocc = occidx.shape[0]
nvir = viridx.shape[0]
nmo = nocc + nvir

e_aibj = lib.direct_sum(
    "n+m-a-b->nbma",
    mo_energy[viridx],
    mo_energy[viridx],
    mo_energy[occidx],
    mo_energy[occidx]
)

d = 1/e_aibj
ao2mo.general(
        mol,
        (mf.mo_coeff, mf.mo_coeff, mf.mo_coeff, mf.mo_coeff),
        f"{nmo}.h5",
        compact=False,
    )


In [ ]:
#from pyppm.soppa import SOPPA
import time
import cupy as cp
import dask.array as da
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import h5py 
import numpy as np



import gc
# Liberar memoria de la GPU
def free_gpu_memory():
    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    gc.collect()



In [ ]:
#Usar preallocated memory pool for GPU as suggested in https://github.com/dask/dask/issues/7687
#De momento esto no es estrictamente necesario. No veo diferencia en la performance.

#import rmm
#from rmm.allocators.cupy import rmm_cupy_allocator

# Set RMM as CuPy's memory allocator
#rmm.reinitialize(
#    pool_allocator=True,
#    initial_pool_size=3*1024**3,  # Adjust this size based on your requirements
#)
#cp.cuda.set_allocator(rmm_cupy_allocator)

In [ ]:


def calculate_energy_differences(mo_energy, occidx, viridx):
    e_aibj = lib.direct_sum(
        "n+m-a-b->nbma",
        mo_energy[viridx],
        mo_energy[viridx],
        mo_energy[occidx],
        mo_energy[occidx]
    )
    return 1 / e_aibj

#def load_integrals(h5_file, nmo, gpu):
#    with h5py.File(h5_file, "r") as f:
#        #eri_mo = da.from_array(f["eri_mo"], chunks=(nmo, nmo))
#        eri_mo = da.from_array(f["eri_mo"], chunks=(200, 200))
#        print("cantidad de nmos:",nmo)
#        eri_mo = eri_mo.reshape(nmo, nmo, nmo, nmo)
#        if (gpu):
#            eri_mo = cp.asarray(eri_mo)
#    return eri_mo

def load_integrals_new(h5_file, nmo, gpu, chunks_inp):
    with h5py.File(h5_file, "r") as f:
        # Load the data into a numpy array first to avoid issues with h5py serialization
        eri_mo_data = np.array(f["eri_mo"])
    eri_mo = da.from_array(eri_mo_data, chunks=chunks_inp)
    print("cantidad de nmos:", nmo)
    eri_mo = eri_mo.reshape(nmo, nmo, nmo, nmo)
    if gpu:
        eri_mo = cp.asarray(eri_mo)
    return eri_mo


def extract_intermediate_integrals(eri_mo, nocc, nvir):
    nmo = nocc + nvir
    int1_ = eri_mo[nocc:, :nocc, nocc:, nocc:]
    int2_ = eri_mo[nocc:, nocc:, nocc:, :nocc]
    int3_ = eri_mo[nocc:, :nocc, :nocc, :nocc]
    int4_ = eri_mo[:nocc, :nocc, nocc:, :nocc]
    return int1_, int2_, int3_, int4_

def calculate_s2_t(mask_ag, mask_bg, mask_np, mask_mp, int1_, int2_, int3_, int4_, deltas,gpu):
    int_mbnp = int1_.transpose(2, 1, 0, 3)
    int_mpnb = int2_.transpose(2, 3, 0, 1)
    int_magb = int3_.transpose(3, 0, 1, 2)
    int_mbga = int3_.transpose(1, 0, 3, 2)
    int_manp = int1_.transpose(2, 0, 1, 3)
    int_mpna = int2_.transpose(2, 0, 3, 1)
    int_gbna = int4_.transpose(2, 1, 3, 0)
    int_ganb = int4_.transpose(2, 3, 1, 0)
    
    c2_1 = int_mbnp - int_mpnb
    c2_2 = int_mpna - int_manp
    c2_3 = int_magb - int_mbga
    c2_4 = int_ganb - int_gbna


    if (gpu):
        s_2 = (
        cp.einsum('ag,nbmp->nbmapg', mask_ag, c2_1) +
        cp.einsum('bg,nmap->nbmapg', mask_bg, c2_2) +
        cp.einsum('np,bmag->nbmapg', mask_np, c2_3) +
        cp.einsum('mp,nbag->nbmapg', mask_mp, c2_4)
        )
        cte = -cp.sqrt(3) / cp.sqrt(2)
        s_2 = cp.einsum('nbma,nbmapg->nbmapg', deltas, cte * s_2)
        result = cp.einsum('nbmapg->pgnbma', s_2)
    else:
        s_2 = (
        da.einsum('ag,nbmp->nbmapg', mask_ag, c2_1) +
        da.einsum('bg,nmap->nbmapg', mask_bg, c2_2) +
        da.einsum('np,bmag->nbmapg', mask_np, c2_3) +
        da.einsum('mp,nbag->nbmapg', mask_mp, c2_4)
        )
        cte = -np.sqrt(3) / np.sqrt(2)
        s_2 = da.einsum('nbma,nbmapg->nbmapg', deltas, cte * s_2)
        result = da.einsum('nbmapg->pgnbma', s_2)
    
    
    return result


def calculate_s1_t(mask_ag, mask_bg, mask_np, mask_mp, int1_, int2_, int3_, int4_, gpu):
    int_mbnp = int1_.transpose(2, 1, 0, 3)
    int_mpnb = int2_.transpose(2, 3, 0, 1)
    int_magb = int3_.transpose(3, 0, 1, 2)
    int_mbga = int3_.transpose(1, 0, 3, 2)
    int_manp = int1_.transpose(2, 0, 1, 3)
    int_mpna = int2_.transpose(2, 0, 3, 1)
    int_gbna = int4_.transpose(2, 1, 3, 0)
    int_ganb = int4_.transpose(2, 3, 1, 0)
    
    c1_1 = int_mbnp + int_mpnb
    c1_2 = int_manp + int_mpna
    c1_3 = int_magb + int_mbga
    c1_4 = int_gbna + int_ganb

    if (gpu):
        s_1 = (
        cp.einsum('ag,nbmp->nbmapg', -mask_ag, c1_1) +
        cp.einsum('bg,nmap->nbmapg', -mask_bg, c1_2) +
        cp.einsum('np,bmag->nbmapg', mask_np, c1_3) +
        cp.einsum('mp,nbag->nbmapg', mask_mp, c1_4)
        )
        s_1 = -s_1 / cp.sqrt(2)
        result = cp.einsum('nbmapg->pgnbma', s_1)
    else:
        s_1 = (
        da.einsum('ag,nbmp->nbmapg', -mask_ag, c1_1) +
        da.einsum('bg,nmap->nbmapg', -mask_bg, c1_2) +
        da.einsum('np,bmag->nbmapg', mask_np, c1_3) +
        da.einsum('mp,nbag->nbmapg', mask_mp, c1_4)
        )
        s_1 = -s_1 / np.sqrt(2)
        result = da.einsum('nbmapg->pgnbma', s_1)

    return result

def compute_da0(s_1_t, s_2_t, d, gpu):
    if(gpu):
        da0 = cp.einsum('pgnbma,nbma,nbmaqd->pgqd', s_1_t, d, s_1_t)
        da0 += cp.einsum('pgnbma,nbma,nbmaqd->pgqd', s_2_t, d, s_2_t)
        da0 = cp.einsum('pgqd->gpdq', da0 / 4)
    else:
        da0 = da.einsum('pgnbma,nbma,nbmaqd->pgqd', s_1_t, d, s_1_t)
        da0 += da.einsum('pgnbma,nbma,nbmaqd->pgqd', s_2_t, d, s_2_t)
        da0 = da.einsum('pgqd->gpdq', da0 / 4)

    #return da0.compute()
    return da0

def main(gpu = False, chunks=(5,5)):
    h5_file = f"{nmo}.h5"
    
    d = calculate_energy_differences(mo_energy, occidx, viridx)
    eri_mo = load_integrals_new(h5_file, nmo, gpu, chunks_inp=chunks)
    int1_, int2_, int3_, int4_ = extract_intermediate_integrals(eri_mo, nocc, nvir)
    #print("Im here -0")
    if (gpu):
        mask_bg = cp.eye(nocc)
        mask_np = cp.eye(nvir)
    else:
        mask_bg = np.eye(nocc)
        mask_np = np.eye(nvir)
    mask_ag = mask_bg
    mask_mp = mask_np

    delta_vir = 1 - mask_np
    delta_occ = 1 - mask_bg

    if (gpu):
        deltas = cp.einsum('nm,ab->nbma', delta_vir, delta_occ)
    else:
        deltas = np.einsum('nm,ab->nbma', delta_vir, delta_occ)
    

    #print("Im here -1")
    s_2_t = calculate_s2_t(mask_ag, mask_bg, mask_np, mask_mp, int1_, int2_, int3_, int4_, deltas, gpu)
    #print("Im here -2")
    s_1_t = calculate_s1_t(mask_ag, mask_bg, mask_np, mask_mp, int1_, int2_, int3_, int4_, gpu)



    if (gpu):
        s_1_t_cupy = cp.asarray(s_1_t)
        s_2_t_cupy = cp.asarray(s_2_t)
        d_cupy = cp.asarray(d)
        da0 = compute_da0(s_1_t_cupy, s_2_t_cupy, d_cupy,gpu)
        print("resultado-gpu:",da0.sum())
        #del s_1_t_cupy
        #del s_2_t_cupy
        #del d_cupy
        #del da0
        #cp._default_memory_pool.free_all_blocks()
    else:
        da0 = compute_da0(s_1_t, s_2_t, d,gpu)
        print("resultado-cpu:",da0.sum().compute())
    
    
mychunks=(5000,5000)    
    
start_gpu_time = time.time()
main(chunks=mychunks, gpu=True)
end_gpu_time = time.time()
print(f"GPU time: {end_gpu_time - start_gpu_time}")

# Llamar a esta función después de tus cálculos en gpu
free_gpu_memory()


start_cpu_time = time.time()
main(chunks=mychunks, gpu=False)
end_cpu_time = time.time()
print(f"CPU time: {end_cpu_time - start_cpu_time}")





#if __name__ == "__main__":
#    main(gpu=True)
